In [2]:
!pip install awswrangler

  Using cached awswrangler-3.4.2-py3-none-any.whl.metadata (23 kB)
  Using cached typing_extensions-4.9.0-py3-none-any.whl.metadata (3.0 kB)
  Using cached urllib3-2.0.7-py3-none-any.whl.metadata (6.6 kB)
Using cached awswrangler-3.4.2-py3-none-any.whl (396 kB)
Using cached typing_extensions-4.9.0-py3-none-any.whl (32 kB)
Using cached urllib3-2.0.7-py3-none-any.whl (124 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.1.0
    Uninstalling urllib3-2.1.0:
      Successfully uninstalled urllib3-2.1.0
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.3.0
    Uninstalling typing_extensions-4.3.0:
      Successfully uninstalled typing_extensions-4.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.3.3 requires pyqt5<5.16, which is not installed.
spyder 5.3.3 requires pyqtwebengine<5.

In [3]:
import pandas as pd
import awswrangler as wr
import numpy as np
from unidecode import unidecode
from itertools import combinations
from datetime import date

In [4]:
def query(query, username):
    """Retorna df da consulta ao banco de dados"""
    
    from datetime import datetime
    dt = datetime.today().strftime('%Y-%m-%d-%H-%M-%S-%f')
    path = f"s3://will-sdx-ml-platform-sagemaker-transient/{username}/{dt}"

    df = wr.athena.read_sql_query(query, 
    database=None, 
    ctas_approach=False, 
    s3_output=path)

    return df

In [5]:
query_string = '''

select
	 id_cliente as id_customer
	,case
		when est.ds_classificacao_nivel_2 = 'SERVICOS' then est.ds_classificacao_nivel_4
		when est.ds_classificacao_nivel_2 = 'SUPERMERCADO' then est.ds_classificacao_nivel_3
		when est.ds_classificacao_nivel_2 = 'LOJAS DE DEPARTAMENTO' then est.ds_classificacao_nivel_4
		when est.ds_classificacao_nivel_2 = 'SERVICOS DE ALIMENTACAO' then est.ds_classificacao_nivel_3
		when est.ds_classificacao_nivel_2 = 'MOBILIDADE' then est.ds_classificacao_nivel_4
		when est.ds_classificacao_nivel_2 = 'E-COMMERCE' then est.ds_classificacao_nivel_3
		when est.ds_classificacao_nivel_2 = 'SAUDE' then est.ds_classificacao_nivel_3
		when est.ds_classificacao_nivel_2 is null then 'c_'||tra.ds_tipo_mov
		when est.ds_classificacao_nivel_2 = '' then 'c_'||tra.ds_tipo_mov
		else est.ds_classificacao_nivel_2
	end as mcc
	, count(*) as frq_uso
from
	backoffice_curated_zone.transaction as tra
 inner join account_sandbox_zone.customer_amostra_crm_rec    as sel on (sel.id_customer = tra.id_cliente)
 left join customer_curated_zone.ca_dict_merchant_classifier as est
 	on (tra.ds_nome_estabelecimento = est.ds_nome_estabelecimento)
 where
 	date(tra.dt_data) >= date('2023-12-12')
 	and ds_empresa = 'Will'
 	--and is_cancelamento = 'Nao'
    --and tra.is_spending = 'Sim'
 	and upper(ds_tipo_mov) in (
    'ANTECIPACAO DE FATURA',
    'ANTECIPAÇÃO FGTS - C CLIENTE',
    'CASHBACK LOJA WILL',
    'COMPRA A VISTA',
    'COMPRA A VISTA INTER',
    'COMPRA GIFT CARD LOJA WILL',
    'COMPRA NO DEBITO',
    'COMPRA PARC LOJ_PARC ASSOC',
    'COMPRA PARCELADA EMISSOR INTERNO',
    'CREDIT VOUCHER',
    'CREDITO CASHBACK',
    'CREDITO EM CONTA',
    'CREDITO PARC FATURA',
    'CREDITO POR TRANFERENCIA',
    'DEPOSITO POR BOLETO',
    'DESC ANTECIP COMPRA',
    'DEVOLUÇÃO PIX',
    'EMPRÉSTIMO PESSOAL',
    'ENTRADA DE PARCELAMENTO',
    'ENVIO PIX CREDITO',
    'FATURA PENDENTE DE PAGAMENTO',
    'MULTA POR ATRASO',
    'PAG FATURA COM SALDO',
    'PAGAMENTO BOLETO',
    'PAGAMENTO DE TITULOS',
    'PAGAMENTO EMPRÉSTIMO PESSOAL',
    'PAGAMENTO FATURA',
    'PAGAMENTO FGTS',
    'PAGAMENTO PARCIAL',
    'PAGAMENTO RECEBIDO',
    'PAGTO DE TITULOS NO CREDITO',
    'PARC. LOJISTA MC (5X)',
    'PARCELAMENTO DE FATURA',
    'PIX ENVIADO',
    'PIX NO CREDITO',
    'PIX RECEBIDO',
    'RECARGA',
    'RECARGA DE CELULAR',
    'RECEBIDO VIA DOC',
    'SAQUE',
    'SAQUE REDE BANDEIRA',
    'TED',
    'TRANSFERENCIA DE SALDO',
    'TRANSFERENCIA ENVIADA',
    'TRANSFERENCIA RECEBIDA'
)
 group by 1,2

'''

In [6]:
transacoes = query(query_string, 'flavia-costa')

In [7]:
transacoes.shape

(429903, 3)

In [8]:
# Contagem da recomendação estimada
#col_est = df.filter(like='e_').columns

In [9]:
def padronizar_mcc(df, col):
    df[col] =  df[col].str.upper() \
                    .apply(lambda x: unidecode(x)) \
                    .str.replace('- GERAL', '') \
                    .str.replace('- C CLIENTE', '') \
                    .apply(lambda x: x.rsplit('-', 1)[-1].strip()) \
                    .str.replace(' / ', '_') \
                    .str.replace(' ', '_')
    
    df[col] = np.where(df[col] == 'TELECOM', 'RECARGA', df[col])
    df[col] = np.where(df[col] == 'NAO_ESPECIFICADO', 'SERVICOS', df[col])
    df[col] = df[col].str.replace(r'.*EMPRESTIMO_PESSOAL.*', 'C_EMPRESTIMO_PESSOAL', regex=True)
    df[col] = df[col].str.replace(r'.*SAQUE.*', 'C_SAQUE', regex=True)
    df[col] = df[col].str.replace(r'.*COMPRA_A_VISTA.*', 'C_COMPRA_PARCELADA', regex=True)
    df[col] = df[col].str.replace(r'.*_PARC.*', 'C_COMPRA_A_VISTA', regex=True)
    df[col] = df[col].str.replace(r'.*FGTS.*', 'C_ANTECIPACAO_FGTS', regex=True)
    df[col] = df[col].str.replace(r'.*RECARGA.*', 'RECARGA', regex=True)
  
  
    return df

In [10]:
# Ajustes do campo de MCC
#
transacoes = padronizar_mcc(transacoes,'mcc')

# Reagrupar os clintes
#
transacoes = transacoes.groupby([transacoes['id_customer'],transacoes['mcc']]).agg({'frq_uso':'sum'}).reset_index()

print(transacoes.shape,"\n ========== \n", transacoes['mcc'].unique())

(429073, 3) 
 ['C_MULTA_POR_ATRASO' 'C_PAGAMENTO_FATURA' 'C_PIX_RECEBIDO'
 'C_COMPRA_NO_DEBITO' 'C_PIX_ENVIADO' 'C_TRANSFERENCIA_RECEBIDA'
 'ENTRETENIMENTO' 'VAREJISTAS' 'MULTICATEGORIA' 'VESTUARIO'
 'C_DEVOLUCAO_PIX' 'C_TRANSFERENCIA_ENVIADA' 'C_COMPRA_A_VISTA'
 'DISTRIBUIDORES' 'LOJAS_ESPECIALIZADAS' 'EDUCACAO' 'C_EMPRESTIMO_PESSOAL'
 'ELETRONICOS' 'RESTAURANTES' 'C_PAGAMENTO_BOLETO' 'RECARGA'
 'APP_TRANSPORTE' 'ATACADISTA' 'BELEZA' 'DELIVERY' 'PADARIAS' 'TI'
 'VIAGEM' 'C_PIX_NO_CREDITO' 'ARMARINHO' 'FARMACIAS_E_DROGARIAS'
 'LANCHONETES' 'SERVICOS' 'FINANCAS' 'C_TRANSFERENCIA_DE_SALDO'
 'COMBUSTIVEL' 'TAXIS' 'DENTISTAS' 'C_SAQUE' 'C_DEPOSITO_POR_BOLETO'
 'C_ANTECIPACAO_FGTS' 'CONSULTA_CREDITO' 'SUPRIMENTOS_PARA_CASA'
 'CONTABILIDADE_AUDITORIA_SEGUROS' 'C_CASHBACK_LOJA_WILL' 'ONIBUS'
 'MANUTENCAO_AUTOMOTIVA' 'ACOUGUE' 'ESPORTIVO' 'MATERIAL_EQUIP_ESCRITORIO'
 'C_CREDITO_POR_TRANFERENCIA' 'PET' 'ELETRICIDADE_GAS'
 'REFORMAS_CONSTRUCAO' 'HOSPITAIS_OU_CLINICAS' 'REPAROS'
 'OUTROS_TRANSP_P

In [11]:
transacoes.head(5)

,id_customer,mcc,frq_uso
0,0000073e-63eb-449d-8843-c96fcfd0e4c7,C_MULTA_POR_ATRASO,1
1,0000a9c3-df5c-4837-b3f4-27af8a3eee9e,C_PAGAMENTO_FATURA,1
2,0000a9c3-df5c-4837-b3f4-27af8a3eee9e,C_PIX_RECEBIDO,1
3,00011f3e-9b75-46a9-9a8c-e654d204db13,C_PAGAMENTO_FATURA,1
4,00011f3e-9b75-46a9-9a8c-e654d204db13,C_PIX_RECEBIDO,1


In [28]:
transacoes['mcc'].value_counts().head(10)

mcc
C_PIX_RECEBIDO        113239
C_PAGAMENTO_FATURA     90287
C_PIX_ENVIADO          70951
C_MULTA_POR_ATRASO     31199
C_COMPRA_NO_DEBITO     24465
VAREJISTAS              8921
VESTUARIO               7315
C_PAGAMENTO_BOLETO      6595
RECARGA                 6415
DISTRIBUIDORES          6023
Name: count, dtype: int64

In [12]:
query_string2 = '''

select * from 
customer_sandbox_zone.tb_crm_rec 

'''

In [13]:
recomend = query(query_string2, 'flavia-costa')

In [14]:
recomend.shape

(9356648, 17)

In [15]:
recomend.columns

Index(['id_customer', 'mcc', 'rec', 'lift', 'tempo_medio', 'day_last_order',
       'vl_abs_order', 'vl_credit_limit', 'rank', 'order_geral', '1dom',
       '2seg', '3ter', '4qua', '5qui', '6sex', '7sab'],
      dtype='object')

In [16]:
recomend.head()

,id_customer,mcc,rec,lift,tempo_medio,day_last_order,vl_abs_order,vl_credit_limit,rank,order_geral,1dom,2seg,3ter,4qua,5qui,6sex,7sab
0,c8c01df9-8fd2-4611-990a-980d4a522a26,OTICAS,True,3.794432,51.0,70.0,290.000,560.0,0.04596,3.0,4.0,4.0,3.0,4.0,3.0,3.0,6.0
1,c8c01df9-8fd2-4611-990a-980d4a522a26,LOJAS_ESPECIALIZADAS,True,1.824029,33.0,194.0,389.030,560.0,0.04424,4.0,2.0,2.0,1.0,1.0,1.0,9.0,9.0
2,c8c01df9-8fd2-4611-990a-980d4a522a26,ESPORTIVO,True,6.813417,47.0,70.0,172.000,560.0,0.04097,5.0,10.0,8.0,9.0,6.0,8.0,4.0,3.0
3,c8c01df9-8fd2-4611-990a-980d4a522a26,C_COMPRA_PARCELADA,True,2.721247,61.0,70.0,284.875,560.0,0.03744,6.0,1.0,1.0,2.0,5.0,5.0,10.0,10.0
4,c8c01df9-8fd2-4611-990a-980d4a522a26,MANUTENCAO_AUTOMOTIVA,True,3.528423,50.0,70.0,140.000,560.0,0.03137,7.0,5.0,7.0,6.0,7.0,6.0,6.0,8.0


In [17]:
print(recomend['mcc'].unique())

<StringArray>
[                         'OTICAS',            'LOJAS_ESPECIALIZADAS',
                       'ESPORTIVO',              'C_COMPRA_PARCELADA',
           'MANUTENCAO_AUTOMOTIVA',                       'ACADEMIAS',
           'SUPRIMENTOS_PARA_CASA',                          'SEGURO',
   'C_PAGTO_DE_TITULOS_NO_CREDITO',                'C_PIX_NO_CREDITO',
                'ELETRICIDADE_GAS',                             'PET',
                      'VAREJISTAS',                        'SERVICOS',
             'REFORMAS_CONSTRUCAO',                         'REPAROS',
                       'BICICLETA',                         'ACOUGUE',
 'CONTABILIDADE_AUDITORIA_SEGUROS',                        'PADARIAS',
                  'DISTRIBUIDORES',            'PUBLICIDADE_ANUNCIOS',
           'HOSPITAIS_OU_CLINICAS',                     'ELETRONICOS',
       'MATERIAL_EQUIP_ESCRITORIO',                 'ESTACIONAMENTOS',
              'C_ANTECIPACAO_FGTS',                       'VEST

In [19]:
recomend_top5 = recomend.query('order_geral <= 5')

In [20]:
recomend_top5['mcc'].value_counts()

mcc
C_PAGTO_DE_TITULOS_NO_CREDITO      806238
C_EMPRESTIMO_PESSOAL               658230
C_COMPRA_PARCELADA                 343563
C_PIX_NO_CREDITO                   303979
LOJAS_ESPECIALIZADAS               237687
OTICAS                             232453
C_DEPOSITO_POR_BOLETO              148682
MULTICATEGORIA                     143638
MANUTENCAO_AUTOMOTIVA              141380
FINANCAS                           136686
C_ANTECIPACAO_FGTS                 130955
HOSPITAIS_OU_CLINICAS              130159
COSMETICOS                         125423
RECARGA                            119910
EDUCACAO                           113777
DISTRIBUIDORES                     107405
BICICLETA                          105120
ELETRICIDADE_GAS                    80539
VESTUARIO                           78888
ESPORTIVO                           76439
BELEZA                              72934
VIAGEM                              72754
REPAROS                             71734
VAREJISTAS                    

In [56]:
len(recomend_top5['id_customer'].unique())

1264400

In [24]:
recomend_top5 = recomend_top5.rename(columns = {'mcc':'mcc_recom'})

In [25]:
join = recomend_top5[['id_customer', 'mcc_recom']].join(transacoes.set_index('id_customer'), on = 'id_customer', how = 'inner')

In [26]:
join.head()

,id_customer,mcc_recom,mcc,frq_uso
123,c8c0d6df-c880-47dc-b8c3-a551919b0f5f,C_EMPRESTIMO_PESSOAL,C_PAGAMENTO_FATURA,1
123,c8c0d6df-c880-47dc-b8c3-a551919b0f5f,C_EMPRESTIMO_PESSOAL,C_PIX_RECEBIDO,2
124,c8c0d6df-c880-47dc-b8c3-a551919b0f5f,ELETRICIDADE_GAS,C_PAGAMENTO_FATURA,1
124,c8c0d6df-c880-47dc-b8c3-a551919b0f5f,ELETRICIDADE_GAS,C_PIX_RECEBIDO,2
125,c8c0d6df-c880-47dc-b8c3-a551919b0f5f,LOJAS_ESPECIALIZADAS,C_PAGAMENTO_FATURA,1


In [46]:
join.groupby('mcc').agg({'id_customer':'count'}).sort_values('id_customer', ascending = False).head(10)

,id_customer
mcc,
C_PIX_RECEBIDO,268326
C_PAGAMENTO_FATURA,212211
C_PIX_ENVIADO,188213
C_COMPRA_NO_DEBITO,82432
C_MULTA_POR_ATRASO,41646
VAREJISTAS,29013
VESTUARIO,22127
RECARGA,20258
C_PAGAMENTO_BOLETO,19568


In [54]:
#Filtrei o top 5 de transações que também possuem recomendação
top_5_geral = ['C_COMPRA_NO_DEBITO', 'C_PAGAMENTO_BOLETO', 'VAREJISTAS', 'VESTUARIO', 'RECARGA']

In [55]:
len(join[join['mcc'].isin(top_5_geral)]['id_customer'].unique())

28471

In [33]:
len(join[join['mcc'] == join['mcc_recom']]['id_customer'].unique())

7096

In [58]:
conversao = pd.DataFrame()

for i in recomend['mcc'].unique():
    join['flag_recom'] = np.where(join['mcc_recom'] == i, 1, 0)
    join['flag_transacao'] = np.where(join['mcc'] == i, 1, 0)
    conversao_tmp = join.groupby('id_customer').agg({'flag_recom':'max', 'flag_transacao':'max'})
    conversao_tmp = conversao_tmp.reset_index()
    conversao_tmp.columns = ['id_customer', 'flag_recom', 'flag_transacao']
    conversao_tmp = conversao_tmp.groupby('flag_recom').agg({'id_customer':'count', 'flag_transacao':'sum'})
    conversao_tmp = conversao_tmp.reset_index()
    conversao_tmp.columns = ['flag_recom', 'clientes', 'conversao']
    conversao_tmp['conversao_perc'] = conversao_tmp['conversao'] / conversao_tmp['clientes']
    conversao_tmp['categoria'] = i
    conversao = pd.concat([conversao, pd.DataFrame(conversao_tmp)], ignore_index=True)
    

In [59]:
conversao

,flag_recom,clientes,conversao,conversao_perc,categoria
0,0,58280,86,0.001476,OTICAS
1,1,14090,28,0.001987,OTICAS
2,0,60019,970,0.016162,LOJAS_ESPECIALIZADAS
3,1,12351,264,0.021375,LOJAS_ESPECIALIZADAS
4,0,67962,72,0.001059,ESPORTIVO
...,...,...,...,...,...
101,1,997,17,0.017051,DELIVERY
102,0,71729,1436,0.020020,APP_TRANSPORTE
103,1,641,19,0.029641,APP_TRANSPORTE
104,0,72343,569,0.007865,C_SAQUE


In [60]:
conversao.to_csv('conversao.csv')